In [ ]:
import pandas as pd
import numpy as np

# Path to place the split data
path = 'data_splits/'

In [ ]:
# Only load these specific columns
columns_to_load = ["DAYOFSERVICE", "TRIPID", "PROGRNUMBER", "STOPPOINTID", "PLANNEDTIME_ARR",
                  "PLANNEDTIME_DEP", "ACTUALTIME_ARR", "ACTUALTIME_DEP", "VEHICLEID", "LASTUPDATE"]

In [79]:
# Load csv in chunks of 1,000,000 rows
df_chunk = pd.read_csv('rt_leavetimes_DB_2018.txt', sep = ";",chunksize=1000000,usecols = columns_to_load, dtype =
                     {"PROGRNUMBER":np.int16, "STOPPOINTID":np.int16, "TRIPID":np.int32,
                      "PLANNEDTIME_ARR":np.int32, "PLANNEDTIME_DEP":np.int32, 
                      "ACTUALTIME_ARR":np.int32, "ACTUALTIME_DEP":np.int32})

In [ ]:
# SQL query to create three seperate dictionaries containing required features from rt_trips_DB_2018
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="Dublinbus",
  passwd="Dublinbus",
  database="Dublinbus",
    port=3306
)

mycursor = mydb.cursor()

mycursor.execute("SELECT tripid,lineid,routeid,direction FROM rt_trips_DB_2018")

myresult = mycursor.fetchall()

trips_dict_route = {}
trips_route_id = {}
trips_direction = {}

for item in myresult:
    trips_dict_route[item[0]] = item[1]
    trips_route_id[item[0]] = item[2]
    trips_direction[item[0]] = item[3]

In [81]:
# Automated chunking of csv
# Addition of required information from the trips data file
file_num = 1
for chunk in df_chunk:
    
    chunk['ROUTE']= chunk['TRIPID'].map(trips_dict_route)
    chunk['ROUTE_ID']= chunk['TRIPID'].map(trips_route_id)
    chunk['DIRECTION']= chunk['TRIPID'].map(trips_direction)
    
    chunk.to_csv(path + 'chunk_data_' + str(file_num) + '.csv', index=False)
    file_num += 1

In [82]:
# Creates a list of all bus routes 
all_routes = []

for item in myresult:
    all_routes.append(item[1])
    
all_routes = set(all_routes)
all_routes = list(all_routes)

In [83]:
# Creates a list of file names for the bus routes
file_name_list = []
for name in range(1,5):
    file_name_list.append('chunk_data_' + str(name) + '.csv')

In [ ]:
# Opens each file in order of name in the list
# For each route, filters the dataframe, appends to a file with the route name
# This is carried out for all data_chunks and routes

for name in file_name_list:
    
    df = pd.read_csv(path + name, sep=",")
    
    for route in all_routes:
        
        df_route = df[df['ROUTE'] == route]
        
        with open(path + 'route_data_files/' + route + '_dataset.csv', 'a') as file:
            df_route.to_csv(file, header = False)